In [2]:
import networkx as nx
import networkx.algorithms.community as nx_comm
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch import optim

def node_connects_cluster(node):
    return set(map(lambda x: inverse_cluster_dict[x], list(g[node]))).union(set([inverse_cluster_dict[node]]))

path = 'Dataset/socfb-Stanford3.mtx'

df = pd.read_table(path, skiprows=1, names = ["source", "target"], sep=" ")
g = nx.from_pandas_edgelist(df)

# calculate basic elements
num_nodes = g.number_of_nodes()
num_edges = g.number_of_edges()
degs = [g.degree[i] for i in g.nodes]
avg_deg = sum(degs)/len(degs)


clu_res = 1
clusters = nx_comm.louvain_communities(g, seed = 10, resolution = clu_res)
clusters = sorted(clusters, key = len, reverse=True)
cluster_sizes = list(map(len, clusters))
num_cluster = len(clusters)

# dict: from node to its cluster
inverse_cluster_dict = {
    node: cl for cl in range(num_cluster) for node in clusters[cl]
}

# dict: from node to its connected cluster
node_to_connected_clusters = {
    node: node_connects_cluster(node) for node in range(1, num_nodes + 1)
}

In [3]:
num_cluster

10

In [4]:
for node in g.nodes:
    cl = inverse_cluster_dict[node]
    cnt = sum([1 for neighbour in g[node] if inverse_cluster_dict[neighbour] == cl]) # in the same cluster
    g.nodes[node]['ai'] = cnt
    g.nodes[node]['bi'] =  g.degree[node] - cnt
    

C_edges = np.zeros((num_cluster, num_cluster), dtype=np.int64)

for edge in g.edges:
    ci = inverse_cluster_dict[edge[0]]
    cj = inverse_cluster_dict[edge[1]]
    C_edges[ci,cj] += 1
    C_edges[cj,ci] += 1
    


C_tensor = torch.tensor(C_edges, dtype = torch.float64)

In [8]:
eps = 1e-6
U = torch.tensor(np.load("optimized_cov/L_matrix_cov_res1_omega1_stf.npy"))
Z = torch.matmul(U, U.T)
V = torch.arcsin(Z/(1+eps))/(2*torch.pi)

V 

tensor([[ 0.2498, -0.2491,  0.1572,  0.1212, -0.1576,  0.1898, -0.2489, -0.1214,
          0.0722, -0.1563],
        [-0.2491,  0.2498, -0.1571, -0.1209,  0.1575, -0.1899,  0.2484,  0.1212,
         -0.0723,  0.1563],
        [ 0.1572, -0.1571,  0.2498,  0.1641, -0.2492,  0.1053, -0.1577, -0.1645,
          0.0256, -0.2476],
        [ 0.1212, -0.1209,  0.1641,  0.2498, -0.1639,  0.0768, -0.1216, -0.2496,
         -0.0199, -0.1648],
        [-0.1576,  0.1575, -0.2492, -0.1639,  0.2498, -0.1056,  0.1581,  0.1642,
         -0.0259,  0.2473],
        [ 0.1898, -0.1899,  0.1053,  0.0768, -0.1056,  0.2498, -0.1894, -0.0771,
          0.0856, -0.1049],
        [-0.2489,  0.2484, -0.1577, -0.1216,  0.1581, -0.1894,  0.2498,  0.1219,
         -0.0717,  0.1568],
        [-0.1214,  0.1212, -0.1645, -0.2496,  0.1642, -0.0771,  0.1219,  0.2498,
          0.0197,  0.1651],
        [ 0.0722, -0.0723,  0.0256, -0.0199, -0.0259,  0.0856, -0.0717,  0.0197,
          0.2498, -0.0240],
        [-0.1563,  

In [10]:
C_tensor.int()

tensor([[207794,  20623,  32982,    834,  32586,   2486,  22251,  10736,     11,
              4],
        [ 20623, 131136,  13012,   3894,  15516,    533,   9655,   4681,      0,
              0],
        [ 32982,  13012, 105658,   1291,  19162,    638,  12088,   2325,      1,
              0],
        [   834,   3894,   1291,  37936,   1531,     97,   1198,    327,      0,
              0],
        [ 32586,  15516,  19162,   1531,  86852,    755,  10855,   5419,      1,
              2],
        [  2486,    533,    638,     97,    755,  10452,    770,    250,      0,
              0],
        [ 22251,   9655,  12088,   1198,  10855,    770,  74794,   2333,      6,
              1],
        [ 10736,   4681,   2325,    327,   5419,    250,   2333,  24214,      0,
              0],
        [    11,      0,      1,      0,      1,      0,      6,      0,     52,
              0],
        [     4,      0,      0,      0,      2,      0,      1,      0,      0,
             22]], dtype=tor

In [14]:
import array_to_latex as a2l

a2l.to_ltx(C_tensor.int().numpy(), frmt = '{:6d}')

\begin{bmatrix}
  207794  &   20623  &   32982  &     834  &   32586  &    2486  &   22251  &   10736  &      11  &       4 \\
   20623  &  131136  &   13012  &    3894  &   15516  &     533  &    9655  &    4681  &       0  &       0 \\
   32982  &   13012  &  105658  &    1291  &   19162  &     638  &   12088  &    2325  &       1  &       0 \\
     834  &    3894  &    1291  &   37936  &    1531  &      97  &    1198  &     327  &       0  &       0 \\
   32586  &   15516  &   19162  &    1531  &   86852  &     755  &   10855  &    5419  &       1  &       2 \\
    2486  &     533  &     638  &      97  &     755  &   10452  &     770  &     250  &       0  &       0 \\
   22251  &    9655  &   12088  &    1198  &   10855  &     770  &   74794  &    2333  &       6  &       1 \\
   10736  &    4681  &    2325  &     327  &    5419  &     250  &    2333  &   24214  &       0  &       0 \\
      11  &       0  &       1  &       0  &       1  &       0  &       6  &       0  &      52

In [19]:
a2l.to_ltx((4*V).numpy(), frmt = '{:5.3f}')

\begin{bmatrix}
  0.999 & -0.996 &  0.629 &  0.485 & -0.630 &  0.759 & -0.995 & -0.486 &  0.289 & -0.625\\
 -0.996 &  0.999 & -0.628 & -0.483 &  0.630 & -0.759 &  0.994 &  0.485 & -0.289 &  0.625\\
  0.629 & -0.628 &  0.999 &  0.657 & -0.997 &  0.421 & -0.631 & -0.658 &  0.102 & -0.991\\
  0.485 & -0.483 &  0.657 &  0.999 & -0.656 &  0.307 & -0.486 & -0.998 & -0.080 & -0.659\\
 -0.630 &  0.630 & -0.997 & -0.656 &  0.999 & -0.423 &  0.632 &  0.657 & -0.104 &  0.989\\
  0.759 & -0.759 &  0.421 &  0.307 & -0.423 &  0.999 & -0.757 & -0.308 &  0.343 & -0.419\\
 -0.995 &  0.994 & -0.631 & -0.486 &  0.632 & -0.757 &  0.999 &  0.487 & -0.287 &  0.627\\
 -0.486 &  0.485 & -0.658 & -0.998 &  0.657 & -0.308 &  0.487 &  0.999 &  0.079 &  0.661\\
  0.289 & -0.289 &  0.102 & -0.080 & -0.104 &  0.343 & -0.287 &  0.079 &  0.999 & -0.096\\
 -0.625 &  0.625 & -0.991 & -0.659 &  0.989 & -0.419 &  0.627 &  0.661 & -0.096 &  0.999
\end{bmatrix}
